In [1]:
import requests, codecs, os, datetime, json, sys

In [14]:
def fetch_search_results(section,zipcode):
    base = 'https://philadelphia.craigslist.org/search/{0}?search_distance=0&postal={1}&format=rss'.format(section,zipcode)
    try:
        resp = requests.get(base, timeout=5,headers={'User-Agent': 'Mozilla/5.0'})
        resp.raise_for_status()
        return resp.content
    except:
        print ("Error fetching RSS for section {0} and zipcode {1}: {2} Continuing...").format(section, zipcode, sys.exc_info())
        pass

In [19]:
def write_results(result, section, subsection, zipcode):
    now = datetime.datetime.now()
    dest = "raw/{0}/{1}/{2}".format(section, subsection, zipcode) + now.strftime('/%Y-%m-%d/%H_%M.xml') 
    try:
        os.makedirs(os.path.dirname(dest))
    except:
        pass
    
    try:
        f = codecs.open(dest,'w')
        f.write(result)
    except:
        print ("Error saving XML for section {0}: {1}. Continuing...").format(section, sys.exc_info()[0])
        pass

In [20]:
def open_json(filename):
    try:
        data = open(filename).read()
        return json.loads(data)
    except:
        raise Exception("Error reading craigslist sections JSON: {0}".format(sys.exc_info()))

In [21]:
sections = open_json('sections.json')
zipcodes = open_json('zips.json')['zipcodes']
for section in sections:
    for subsection in sections[section].values():
        for zipcode in zipcodes:
            result = fetch_search_results(subsection, zipcode)
            write_results(result, section, subsection, zipcode)